In [1]:
# ============================================================
# 11번: 최종 파이프라인 - 새 파생변수 + OOF Target Encoding + LGB+CAT
# ============================================================

import os, gc, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import catboost as cb

# -------------------------
# Config
# -------------------------
DATA_DIR = "../data"
OUT_DIR  = "../outputs"
os.makedirs(OUT_DIR, exist_ok=True)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"
N_FOLDS = 5
SEED = 42

# -------------------------
# 1) Feature Engineering - 기존 + 신규 발견 피처 전부 포함
# -------------------------
def preprocess(df):
    d = df.copy()

    # ===== 기존 파생변수 (03번 기반) =====
    
    # 시술_대분류
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        if "IUI" in x: return "IUI"
        if "DI" in x:  return "Other"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        return "Other"
    d["시술_대분류"] = d["특정 시술 유형"].apply(major_procedure)

    # BLASTOCYST / AH 포함
    d["BLASTOCYST_포함"] = d["특정 시술 유형"].astype(str).str.contains("BLASTOCYST", na=False).astype(int)
    d["AH_포함"] = d["특정 시술 유형"].astype(str).str.contains("AH", na=False).astype(int)

    # 배아 이식 여부
    embryo_stage_cols = [
        "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유",
        "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "이식된 배아 수", "미세주입 배아 이식 수", "저장된 배아 수",
        "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수",
        "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수",
        "파트너 정자와 혼합된 난자 수", "기증자 정자와 혼합된 난자 수",
        "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부",
    ]
    d["배아_이식_여부"] = 1 - d[embryo_stage_cols].isna().all(axis=1).astype(int)

    # 배아 진행 단계
    def embryo_stage(row):
        if row["배아_이식_여부"] == 0:
            return "배아단계_미도달"
        elif pd.isna(row["총 생성 배아 수"]) or row["총 생성 배아 수"] == 0:
            return "배아생성_실패"
        elif pd.isna(row["이식된 배아 수"]) or row["이식된 배아 수"] == 0:
            return "이식_미실시"
        else:
            return "이식_완료"
    d["배아_진행_단계"] = d.apply(embryo_stage, axis=1)

    # 총시술_bin3
    def collapse_trials(x):
        if x == "0회": return "0회"
        elif x in ["1회","2회"]: return "1–2회"
        else: return "3회 이상"
    d["총시술_bin3"] = d["총 시술 횟수"].apply(collapse_trials)

    # 나이_3구간
    def age_group_simple(age):
        if age == "알 수 없음": return "Unknown"
        elif age == "만18-34세": return "34세 이하"
        elif age in ["만35-37세","만38-39세"]: return "35-39세"
        else: return "40세 이상"
    d["나이_3구간"] = d["시술 당시 나이"].apply(age_group_simple)

    # 이식배아_구간
    def embryo_count_bin(count):
        if pd.isna(count) or count == 0: return "0개"
        elif count <= 2: return "1-2개"
        else: return "3개 이상"
    d["이식배아_구간"] = d["이식된 배아 수"].apply(embryo_count_bin)

    # Day5_이식_여부
    d["Day5_이식_여부"] = (d["배아 이식 경과일"] == 5.0).astype(int)

    # 불임원인 관련
    infertility_cols = [
        "남성 주 불임 원인", "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인",
        "부부 주 불임 원인", "부부 부 불임 원인", "불명확 불임 원인",
        "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", "불임 원인 - 배란 장애",
        "불임 원인 - 여성 요인", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증",
        "불임 원인 - 정자 농도", "불임 원인 - 정자 면역학적 요인", "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태"
    ]
    d["불임_원인_개수"] = d[infertility_cols].sum(axis=1)

    def infertility_complexity(count):
        if count == 0: return "None"
        elif count == 1: return "Single"
        elif count == 2: return "Double"
        else: return "Multiple"
    d["불임원인_복잡도"] = d["불임_원인_개수"].apply(infertility_complexity)

    # 배아 해동 실시 여부
    d["배아_해동_실시_여부"] = d["배아 해동 경과일"].notna().astype(int)

    # 비율
    d["배아_생성_효율"] = d["총 생성 배아 수"] / (d["수집된 신선 난자 수"] + 1)
    d["배아_이식_비율"] = d["이식된 배아 수"] / (d["총 생성 배아 수"] + 1)
    d["배아_저장_비율"] = d["저장된 배아 수"] / (d["총 생성 배아 수"] + 1)

    # 교호작용
    d["나이×Day5"] = d["시술 당시 나이"].astype(str) + "_" + d["Day5_이식_여부"].astype(str)
    d["시술횟수×나이"] = d["총시술_bin3"].astype(str) + "_" + d["나이_3구간"].astype(str)

    # ===== 신규 파생변수 (데이터 분석에서 발견) =====
    
    # ★★★ [핵심1] 배아 생성 주요 이유 → 현재 시술용 포함 여부
    # 현재시술용 미포함 시 성공률 4.4%, 포함 시 27.5% → 극단적 차이
    d["현재시술용_포함"] = d["배아 생성 주요 이유"].str.contains("현재 시술용", na=False).astype(int)
    
    # 저장/기증만 목적 (성공률 0.06%)
    storage_only = ["배아 저장용", "난자 저장용", "기증용", "기증용, 배아 저장용", 
                    "기증용, 난자 저장용", "난자 저장용, 배아 저장용", 
                    "난자 저장용, 배아 저장용, 연구용"]
    d["저장기증만_목적"] = d["배아 생성 주요 이유"].isin(storage_only).astype(int)
    
    # 배아 생성 이유 대분류 (더 세밀하게)
    def embryo_reason_group(x):
        if pd.isna(x): return "미도달"
        if x == "현재 시술용": return "현재시술"
        if "현재 시술용" in x and "기증" in x: return "기증+시술"
        if "현재 시술용" in x: return "시술+기타"
        return "저장기증만"
    d["배아생성이유_대분류"] = d["배아 생성 주요 이유"].apply(embryo_reason_group)

    # ★★★ [핵심2] 배양기간 (배아 이식 경과일 - 난자 혼합 경과일)
    d["배양기간"] = d["배아 이식 경과일"] - d["난자 혼합 경과일"]
    
    # 배양기간 구간화
    def culture_period_bin(x):
        if pd.isna(x): return "Missing"
        elif x >= 5: return "5일이상"  # 40.5%
        elif x >= 3: return "3-4일"    # 25-35%
        else: return "2일이하"          # ~21%
    d["배양기간_구간"] = d["배양기간"].apply(culture_period_bin)

    # ★★★ [핵심3] 시술 횟수 ordinal + 차이 피처
    ord_map = {"0회":0, "1회":1, "2회":2, "3회":3, "4회":4, "5회":5, "6회 이상":6}
    d["총시술_ord"] = d["총 시술 횟수"].map(ord_map)
    d["IVF시술_ord"] = d["IVF 시술 횟수"].map(ord_map)
    d["클리닉시술_ord"] = d["클리닉 내 총 시술 횟수"].map(ord_map)
    d["DI시술_ord"] = d["DI 시술 횟수"].map(ord_map)
    
    # 외부 시술 경험 (다른 클리닉에서 시술한 적 있는지)
    d["외부시술_경험"] = d["총시술_ord"] - d["클리닉시술_ord"]
    # 비IVF 시술 경험 (DI 등 다른 시술)
    d["비IVF시술_경험"] = d["총시술_ord"] - d["IVF시술_ord"]

    # 임신/출산 이력 ordinal
    d["총임신_ord"] = d["총 임신 횟수"].map(ord_map)
    d["IVF임신_ord"] = d["IVF 임신 횟수"].map(ord_map)
    d["총출산_ord"] = d["총 출산 횟수"].map(ord_map)
    d["IVF출산_ord"] = d["IVF 출산 횟수"].map(ord_map)
    
    # ★★★ [핵심4] 과거 성공 이력
    d["과거_임신_경험"] = (d["총임신_ord"] > 0).astype(int)
    d["과거_출산_경험"] = (d["총출산_ord"] > 0).astype(int)
    d["과거_IVF_임신"] = (d["IVF임신_ord"] > 0).astype(int)
    d["과거_IVF_출산"] = (d["IVF출산_ord"] > 0).astype(int)

    # ★★★ [핵심5] 미세주입 관련 비율 (corr 0.076~0.095)
    d["미세주입_생성_효율"] = d["미세주입에서 생성된 배아 수"] / (d["미세주입된 난자 수"] + 1)
    d["미세주입_이식_비율"] = d["미세주입 배아 이식 수"] / (d["이식된 배아 수"] + 0.1)
    d["난자_활용률"] = d["혼합된 난자 수"] / (d["수집된 신선 난자 수"] + 1)
    
    # 기증자 정자 비율
    d["기증자정자_비율"] = d["기증자 정자와 혼합된 난자 수"] / (d["혼합된 난자 수"] + 1)
    
    # 해동 대비 저장 비율
    d["해동_저장_비율"] = d["해동된 배아 수"] / (d["저장된 배아 수"] + 1)
    
    # 저장된 신선 난자 비율
    d["신선난자_저장_비율"] = d["저장된 신선 난자 수"] / (d["수집된 신선 난자 수"] + 1)

    # ★★★ [핵심6] 추가 교호작용
    d["나이×배아진행"] = d["시술 당시 나이"].astype(str) + "_" + d["배아_진행_단계"]
    d["나이×단일이식"] = d["시술 당시 나이"].astype(str) + "_" + d["단일 배아 이식 여부"].fillna(-1).astype(int).astype(str)
    d["시술대분류×Day5"] = d["시술_대분류"] + "_" + d["Day5_이식_여부"].astype(str)
    d["나이×현재시술용"] = d["나이_3구간"] + "_" + d["현재시술용_포함"].astype(str)
    d["배아진행×불임복잡도"] = d["배아_진행_단계"] + "_" + d["불임원인_복잡도"]
    d["시기코드×나이"] = d["시술 시기 코드"] + "_" + d["나이_3구간"]
    
    # ★★★ [핵심7] 난자 기증자 나이 숫자화
    egg_donor_map = {"만20세 이하": 0, "만21-25세": 1, "만26-30세": 2, "만31-35세": 3, "알 수 없음": -1}
    d["난자기증자나이_ord"] = d["난자 기증자 나이"].map(egg_donor_map)
    
    # 난자 기증 여부 (출처가 기증이면)
    d["난자_기증_여부"] = (d["난자 출처"] == "기증 제공").astype(int)

    # ★★★ [핵심8] 결측 패턴 카운트 (전체)
    d["전체_결측_수"] = d.isnull().sum(axis=1)
    
    # 경과일 변수 결측 패턴
    day_cols = ["난자 채취 경과일", "난자 혼합 경과일", "배아 이식 경과일", "배아 해동 경과일", "난자 해동 경과일"]
    d["경과일_결측_수"] = d[day_cols].isnull().sum(axis=1)
    d["난자채취_실시"] = d["난자 채취 경과일"].notna().astype(int)
    d["난자혼합_실시"] = d["난자 혼합 경과일"].notna().astype(int)
    
    # 결측 flag 추가
    d["PGD_실시"] = d["PGD 시술 여부"].notna().astype(int)
    d["PGS_실시"] = d["PGS 시술 여부"].notna().astype(int)
    d["배아이식경과일_isna"] = d["배아 이식 경과일"].isna().astype(int)
    
    # ★★★ [핵심9] 동결/신선 배아 조합
    def embryo_type(row):
        frozen = row["동결 배아 사용 여부"]
        fresh = row["신선 배아 사용 여부"]
        if pd.isna(frozen): return "미도달"
        if frozen == 1 and fresh == 0: return "동결만"
        if frozen == 0 and fresh == 1: return "신선만"
        if frozen == 1 and fresh == 1: return "동결+신선"
        return "둘다아님"
    d["배아_유형"] = d.apply(embryo_type, axis=1)

    return d


# -------------------------
# 2) OOF Target Encoding (leak-free)
# -------------------------
def oof_target_encode(train_df, test_df, col, target_col, n_folds=5, seed=42, smoothing=20):
    """
    OOF 방식 target encoding: train은 fold-out 방식, test는 전체 train 기반.
    smoothing: global mean으로 shrinkage.
    """
    global_mean = train_df[target_col].mean()
    
    # Train: OOF
    train_enc = pd.Series(np.nan, index=train_df.index, dtype=float)
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
    
    for tr_idx, va_idx in skf.split(train_df, train_df[target_col]):
        tr = train_df.iloc[tr_idx]
        stats = tr.groupby(col)[target_col].agg(["mean", "count"])
        stats["enc"] = (stats["mean"] * stats["count"] + global_mean * smoothing) / (stats["count"] + smoothing)
        mapping = stats["enc"].to_dict()
        train_enc.iloc[va_idx] = train_df.iloc[va_idx][col].map(mapping).fillna(global_mean)
    
    # Test: 전체 train 기반
    stats = train_df.groupby(col)[target_col].agg(["mean", "count"])
    stats["enc"] = (stats["mean"] * stats["count"] + global_mean * smoothing) / (stats["count"] + smoothing)
    mapping = stats["enc"].to_dict()
    test_enc = test_df[col].map(mapping).fillna(global_mean)
    
    return train_enc.values, test_enc.values


# -------------------------
# 3) Main Pipeline
# -------------------------
def main():
    print("=" * 80)
    print("11번: 최종 파이프라인 - 새 파생변수 + OOF TE + LGB+CAT")
    print("=" * 80)

    # Load
    train_raw = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
    test_raw  = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
    sub       = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
    
    # Preprocess
    train_df = preprocess(train_raw)
    test_df  = preprocess(test_raw)
    
    y = train_df[TARGET_COL].astype(int).values
    
    # Drop cols
    drop_cols = [ID_COL, TARGET_COL]
    
    # ★ OOF Target Encoding 적용할 카테고리 컬럼들
    te_cols = [
        "시술 시기 코드",          # 7 categories, 시간/클리닉 proxy
        "특정 시술 유형",           # 24 categories
        "시술 당시 나이",           # 7 categories
        "배아 생성 주요 이유",      # 13 categories
        "배아_진행_단계",           # 4 categories
        "나이×Day5",              # 교차
        "시술횟수×나이",           # 교차
        "나이×배아진행",           # 교차
        "시기코드×나이",           # 교차
        "배란 유도 유형",          # 4 categories
        "총 시술 횟수",            # 7 categories
        "클리닉 내 총 시술 횟수",   # 7 categories
        "배아_유형",               # 동결/신선 조합
        "배아생성이유_대분류",      # 5 categories
        "난자 출처",               # 3 categories
        "난자 기증자 나이",         # 5 categories
    ]
    
    print("\nApplying OOF Target Encoding...")
    for col in te_cols:
        if col not in train_df.columns:
            print(f"  [SKIP] {col} not found")
            continue
        new_col = f"TE_{col}"
        tr_enc, te_enc = oof_target_encode(train_df, test_df, col, TARGET_COL, 
                                            n_folds=N_FOLDS, seed=SEED, smoothing=20)
        train_df[new_col] = tr_enc
        test_df[new_col] = te_enc
        print(f"  [OK] {new_col}")
    
    # Feature columns
    feature_cols = [c for c in train_df.columns if c not in drop_cols]
    
    # cat/num 구분
    cat_cols = []
    num_cols = []
    for c in feature_cols:
        if str(train_df[c].dtype) in ["object", "category"]:
            cat_cols.append(c)
        else:
            num_cols.append(c)
    
    print(f"\nTotal features: {len(feature_cols)} (cat={len(cat_cols)}, num={len(num_cols)})")
    print(f"Target rate: {y.mean():.4f}")
    
    # -------------------------
    # LightGBM
    # -------------------------
    print("\n" + "=" * 60)
    print("LightGBM Training")
    print("=" * 60)
    
    lgb_params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "num_leaves": 63,
        "learning_rate": 0.03,
        "feature_fraction": 0.7,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "min_child_samples": 50,
        "lambda_l1": 0.1,
        "lambda_l2": 1.0,
        "verbose": -1,
        "seed": SEED,
        "n_jobs": -1,
    }
    
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
    
    oof_lgb = np.zeros(len(train_df))
    test_lgb = np.zeros(len(test_df))
    
    # LightGBM용 데이터 준비 (cat cols를 category type으로)
    X_train_lgb = train_df[feature_cols].copy()
    X_test_lgb  = test_df[feature_cols].copy()
    for c in cat_cols:
        X_train_lgb[c] = X_train_lgb[c].astype("category")
        X_test_lgb[c]  = X_test_lgb[c].astype("category")
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_train_lgb, y), 1):
        X_tr = X_train_lgb.iloc[tr_idx]
        X_va = X_train_lgb.iloc[va_idx]
        y_tr = y[tr_idx]
        y_va = y[va_idx]
        
        dtrain = lgb.Dataset(X_tr, label=y_tr, categorical_feature=cat_cols)
        dvalid = lgb.Dataset(X_va, label=y_va, categorical_feature=cat_cols, reference=dtrain)
        
        model = lgb.train(
            lgb_params,
            dtrain,
            num_boost_round=3000,
            valid_sets=[dvalid],
            callbacks=[
                lgb.early_stopping(100),
                lgb.log_evaluation(200),
            ]
        )
        
        oof_lgb[va_idx] = model.predict(X_va)
        test_lgb += model.predict(X_test_lgb) / N_FOLDS
        
        fold_auc = roc_auc_score(y_va, oof_lgb[va_idx])
        print(f"  Fold {fold}: AUC = {fold_auc:.6f} (best_iter={model.best_iteration})")
    
    lgb_auc = roc_auc_score(y, oof_lgb)
    print(f"\n>>> LightGBM OOF AUC = {lgb_auc:.6f}")
    
    # Feature importance 출력
    fi = pd.DataFrame({
        "feature": feature_cols,
        "importance": model.feature_importance(importance_type="gain")
    }).sort_values("importance", ascending=False)
    print("\n=== Top 30 Features (LGB) ===")
    print(fi.head(30).to_string())
    
    # -------------------------
    # CatBoost
    # -------------------------
    print("\n" + "=" * 60)
    print("CatBoost Training")
    print("=" * 60)
    
    # CatBoost용 데이터: cat cols를 string으로
    X_train_cb = train_df[feature_cols].copy()
    X_test_cb  = test_df[feature_cols].copy()
    for c in cat_cols:
        X_train_cb[c] = X_train_cb[c].astype(str).fillna("NA")
        X_test_cb[c]  = X_test_cb[c].astype(str).fillna("NA")
    # NaN in numeric
    X_train_cb[num_cols] = X_train_cb[num_cols].fillna(-999)
    X_test_cb[num_cols]  = X_test_cb[num_cols].fillna(-999)
    
    cat_indices = [feature_cols.index(c) for c in cat_cols]
    
    oof_cb = np.zeros(len(train_df))
    test_cb = np.zeros(len(test_df))
    
    for fold, (tr_idx, va_idx) in enumerate(skf.split(X_train_cb, y), 1):
        X_tr = X_train_cb.iloc[tr_idx]
        X_va = X_train_cb.iloc[va_idx]
        y_tr = y[tr_idx]
        y_va = y[va_idx]
        
        pool_tr = cb.Pool(X_tr, y_tr, cat_features=cat_indices)
        pool_va = cb.Pool(X_va, y_va, cat_features=cat_indices)
        
        model_cb = cb.CatBoostClassifier(
            iterations=3000,
            learning_rate=0.03,
            depth=8,
            l2_leaf_reg=3,
            random_seed=SEED,
            eval_metric="AUC",
            task_type="CPU",
            verbose=200,
            early_stopping_rounds=100,
            use_best_model=True,
        )
        
        model_cb.fit(pool_tr, eval_set=pool_va)
        
        oof_cb[va_idx] = model_cb.predict_proba(X_va)[:, 1]
        test_cb += model_cb.predict_proba(X_test_cb)[:, 1] / N_FOLDS
        
        fold_auc = roc_auc_score(y_va, oof_cb[va_idx])
        print(f"  Fold {fold}: AUC = {fold_auc:.6f}")
    
    cb_auc = roc_auc_score(y, oof_cb)
    print(f"\n>>> CatBoost OOF AUC = {cb_auc:.6f}")
    
    # -------------------------
    # Ensemble (최적 weight 탐색)
    # -------------------------
    print("\n" + "=" * 60)
    print("Ensemble Weight Search")
    print("=" * 60)
    
    best_w = 0.5
    best_ens_auc = 0
    for w in np.arange(0.0, 1.01, 0.05):
        oof_ens = w * oof_lgb + (1 - w) * oof_cb
        auc = roc_auc_score(y, oof_ens)
        if auc > best_ens_auc:
            best_ens_auc = auc
            best_w = w
        if abs(w - 0.5) < 0.01 or auc > best_ens_auc - 0.001:
            print(f"  w_lgb={w:.2f}: AUC = {auc:.6f}")
    
    print(f"\n>>> Best Ensemble: w_lgb={best_w:.2f}, OOF AUC = {best_ens_auc:.6f}")
    
    # 최종 예측
    test_ens = best_w * test_lgb + (1 - best_w) * test_cb
    
    # -------------------------
    # Rank Average도 시도
    # -------------------------
    from scipy.stats import rankdata
    oof_rank = (rankdata(oof_lgb) + rankdata(oof_cb)) / 2
    test_rank = (rankdata(test_lgb) + rankdata(test_cb)) / 2
    rank_auc = roc_auc_score(y, oof_rank)
    print(f">>> Rank Average OOF AUC = {rank_auc:.6f}")
    
    # 최고 성능 선택
    results = {
        "lgb_only": (oof_lgb, test_lgb, lgb_auc),
        "cb_only": (oof_cb, test_cb, cb_auc),
        f"ens_w{best_w:.2f}": (best_w * oof_lgb + (1-best_w) * oof_cb, test_ens, best_ens_auc),
        "rank_avg": (oof_rank, test_rank, rank_auc),
    }
    
    best_name = max(results, key=lambda k: results[k][2])
    best_oof, best_test, best_auc_val = results[best_name]
    print(f"\n★ BEST: {best_name} → OOF AUC = {best_auc_val:.6f}")
    
    # -------------------------
    # Save
    # -------------------------
    # 모든 결과 저장
    for name, (oof_arr, test_arr, auc_val) in results.items():
        # submission
        sub2 = sub.copy()
        if "rank" in name:
            # rank는 확률로 변환 (min-max)
            sub2["probability"] = (test_arr - test_arr.min()) / (test_arr.max() - test_arr.min())
        else:
            sub2["probability"] = test_arr
        sub_path = os.path.join(OUT_DIR, f"11_{name}_CV{auc_val:.6f}.csv")
        sub2.to_csv(sub_path, index=False)
        print(f"  Saved: {sub_path}")
    
    # OOF 저장 (나중에 스태킹용)
    np.save(os.path.join(OUT_DIR, "11_oof_lgb.npy"), oof_lgb)
    np.save(os.path.join(OUT_DIR, "11_oof_cb.npy"), oof_cb)
    np.save(os.path.join(OUT_DIR, "11_test_lgb.npy"), test_lgb)
    np.save(os.path.join(OUT_DIR, "11_test_cb.npy"), test_cb)
    
    # Summary
    summary_path = os.path.join(OUT_DIR, "11_summary.txt")
    with open(summary_path, "w", encoding="utf-8") as f:
        f.write("11번 최종 파이프라인 결과\n")
        f.write("=" * 60 + "\n")
        f.write(f"LightGBM OOF AUC: {lgb_auc:.6f}\n")
        f.write(f"CatBoost OOF AUC: {cb_auc:.6f}\n")
        f.write(f"Ensemble (w={best_w:.2f}) OOF AUC: {best_ens_auc:.6f}\n")
        f.write(f"Rank Average OOF AUC: {rank_auc:.6f}\n")
        f.write(f"\nBest: {best_name} → {best_auc_val:.6f}\n")
        f.write(f"\nTotal features: {len(feature_cols)}\n")
        f.write(f"Cat features: {len(cat_cols)}\n")
        f.write(f"Num features: {len(num_cols)}\n")
        f.write(f"\n=== Top 30 Features ===\n")
        f.write(fi.head(30).to_string())
    print(f"  Saved: {summary_path}")
    
    print("\n" + "=" * 80)
    print("DONE!")
    print("=" * 80)


if __name__ == "__main__":
    main()

11번: 최종 파이프라인 - 새 파생변수 + OOF TE + LGB+CAT

Applying OOF Target Encoding...
  [OK] TE_시술 시기 코드
  [OK] TE_특정 시술 유형
  [OK] TE_시술 당시 나이
  [OK] TE_배아 생성 주요 이유
  [OK] TE_배아_진행_단계
  [OK] TE_나이×Day5
  [OK] TE_시술횟수×나이
  [OK] TE_나이×배아진행
  [OK] TE_시기코드×나이
  [OK] TE_배란 유도 유형
  [OK] TE_총 시술 횟수
  [OK] TE_클리닉 내 총 시술 횟수
  [OK] TE_배아_유형
  [OK] TE_배아생성이유_대분류
  [OK] TE_난자 출처
  [OK] TE_난자 기증자 나이

Total features: 141 (cat=37, num=104)
Target rate: 0.2583

LightGBM Training
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.736978
Early stopping, best iteration is:
[150]	valid_0's auc: 0.737096
  Fold 1: AUC = 0.737096 (best_iter=150)
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.741898
Early stopping, best iteration is:
[268]	valid_0's auc: 0.742011
  Fold 2: AUC = 0.742011 (best_iter=268)
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.740119
Early stopping, best iteration is:
[244]	valid_0's auc: